In [1]:
import subprocess
import neutralb1.utils

WORKSPACE_DIR = neutralb1.utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

eb9a1af668e0af18156f9dcb8c01af78f742f8d6


**Repository Version** 
This notebook was run at commit:
`FILL WHEN DONE`

# Verifying the Projection of Moments with Signal MC
As found previously, the projected moments have some missing factors causing them not to match expectations. Since then, two major updates have occured:
1. I've developed a direct fit to data using moments as AmpTools parameters, which should now provide a set of "true" values we can compare the projected moments to.
   <br>a. Has issues in extracting moments with $>1\%$ contribution, but this should be enough to track down factors
2. The old python projection script has now been replaced by a c++ version, that also includes the necessary normalization integrals
   a. The script may likely be updated over time, so check the commit hash for what version to use.

This study will proceed as follows:
1. Generate Signal Monte Carlo (MC) according to a pseudo-realistic set of waves (no acceptance effects i.e. *thrown*)
   <br>a. 35% polarization and in the PARA_0 orientation
2. Fit MC with same waveset, and obtain a fit result that should match the generated wave values
3. Project moments from the fit result to obtain a projected moment-set $H_{\text{proj}}$
4. Fit MC with the same number of moments, and obtain a fitted moment-set $H_{\text{fit}}$
5. Compare the fit and projected sets to investigate the missing factors.

## Setup

In [2]:
# load common libraries
import pandas as pd
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt

import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/input-output-tests/verify-moment/"

# set env variables for shell cells
os.environ['STUDY_DIR'] = STUDY_DIR


## Data Creation

### Generate
We'll use the same cfg file to generate and fit the amplitude-based Monte Carlo with. This will be done in a single bin of mass at:
* $0.1 < -t < 0.2$
* $8.2 < E_\gamma < 8.8$
* $1.20 < M_{\omega\pi^0} < 1.22$

In [3]:
with open(f"{STUDY_DIR}/cfg_files/amplitudes.cfg", "r") as f:
    for i in range(18):
        print(f.readline(), end="")

# This cfg file is designed to generate and fit waves in a single mass bin
# to confirm the projection of moments is working correctly. No acceptance effects
# will be applied, and the phasespace will contain the same TEM cuts as the signal
#   -t: 0.1 - 0.2
#    M: 1.2 - 1.22
#    E: 8.2 - 8.8
#
# The waveset is pseudo-realistic, and comes from the previous truth-initialized fits,
# but has already applied the Breit-Wigner and scale parameter to the complex values


# generator-level configuration parameters
define vector 0.782 0.008
define beamconfig beam.cfg

# fit setup
fit omegapi
normintfile omegapi omegapi.ni


In [4]:
%%bash 
if [ -e "${STUDY_DIR}/data/root_files/data.root" ]; then
    echo "data exists, skipping generation."
else 
    echo "Generating data..."
    gen_vec_ps -c ${STUDY_DIR}/cfg_files/amplitudes.cfg\
        -o ${STUDY_DIR}/data/root_files/data.root\
        -l 1.20 -u 1.22\
        -n 50000\
        -a 8.2 -b 8.8\
        -tmin 0.1 -tmax 0.2
    if [ -e "${STUDY_DIR}/data/root_files/data.root" ]; then
        echo "Data generation successful."
    else
        echo "Data generation failed."
        exit 1
    fi
fi

data exists, skipping generation.


### Fitting

#### Amplitudes
Amplitude fits will require a GPU session due to their performance requirements.

In [9]:
%%bash
if [ -e "${STUDY_DIR}/data/amplitude_results/omegapi.fit" ]; then
    echo "Amplitude results exist, skipping fitting."
else
    echo "Run 'fit -c ${STUDY_DIR}/cfg_files/amplitudes.cfg -m 10000000 -r 50 > amplitude_fit.log' on an interactive GPU node to fit the data."
fi

Run 'fit -c /w/halld-scshelf2101/kscheuer/neutralb1/studies/input-output-tests/verify-moment//cfg_files/amplitudes.cfg -m 10000000 -r 50 > amplitude_fit.log' on an interactive GPU node to fit the data.


In [ ]:
%%bash
# TODO: split into cells
# vecps_plotter omegapi.fit    
# $WORKSPACE_DIR/submission/batch_scripts/angle_plotter vecps_plot.root "Thrown MC" "" ./ true
# python $WORKSPACE_DIR/submission/batch_scripts/convert_to_csv.py -i omegapi.fit -o result.csv
# python $WORKSPACE_DIR/submission/batch_scripts/project_moments.py\
#     -i omegapi.fit -o projected_moments.csv -m 1.21

# mkdir -p amplitude_results
# mv omegapi_*.fit amplitude_results/
# mv omegapi.fit amplitude_results/
# mv omegapi.ni amplitude_results/
# mv vecps_plot.root amplitude_results/
# mv vecps_fitPars.txt amplitude_results/
# mv result.csv amplitude_results/
# mv projected_moments.csv amplitude_results/

# # FIT WITH MOMENTS
# fit -c moments.cfg -m 10000000 -r 50 > moment_fit.log
# if [ -e "omegapi.fit" ]; then
#     echo "Fit successful."    
# else
#     echo "Fit failed."
#     exit 1
# fi
# vecps_plotter omegapi.fit
# $WORKSPACE_DIR/submission/batch_script/angle_plotter vecps_plot.root "Thrown MC" "" ./ true
# python $WORKSPACE_DIR/submission/batch_scripts/convert_to_csv.py -i omegapi.fit -o result.csv

# mkdir -p moment_results
# mv omegapi_*.fit moment_results/
# mv omegapi.fit moment_results/
# mv omegapi.ni moment_results/
# mv vecps_plot.root moment_results/
# mv vecps_fitPars.txt moment_results/
# mv result.csv moment_results/

#### Moments